In [1]:
%load_ext autoreload
%autoreload 2
import json
import os
import sys

import keras.backend as K
from keras.models import load_model
from keras.optimizers import Adam
import numpy as np
import tensorflow as tf

from dataset_utils import adni_loader
#from networks import transform_network_utils

sys.path.append('../neuron')
sys.path.append('../voxelmorph')
import src.losses as vm_losses

gpu_ids = [1]
# set gpu id and tf settings
os.environ['CUDA_VISIBLE_DEVICES']=','.join([str(g) for g in gpu_ids])
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.tensorflow_backend.set_session(tf.Session(config=config))

import sys
sys.path.append('../voxelmorph-sandbox')
import voxelmorph.networks as vm_networks
import tensorflow as tf
from voxelmorph import dense_3D_spatial_transformer
from keras.models import load_model
 
sys.path.append('../neuron')
import neuron.layers as nrn_layers
import neuron.utils as nrn_utils
sys.path.append('../voxelmorph-sandbox')
import voxelmorph.networks as vm_networks
from voxelmorph.dense_3D_spatial_transformer import Dense3DSpatialTransformer


# voxelmorph_model = load_model(
#     './experiments/voxelmorph/vm2_cc_AtoUMS_newdataset_100k_bidir_iter{}.h5'.format(start_iter),
#     custom_objects={'Dense3DSpatialTransformer': dense_3D_spatial_transformer.Dense3DSpatialTransformer, 
#                     'interp_upsampling': vm_networks.interp_upsampling,
#                     'meshgrid': vm_networks.meshgrid,
#                     'tf': tf,
                    
#                     'VecInt': nrn_layers.VecInt,
#                     'SpatialTransformer': nrn_layers.SpatialTransformer,
#                     'nrn_utils': nrn_utils,
#                     'nrn_layers': nrn_layers,
#                    },
#     compile=False,
# )

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "/afs/csail.mit.edu/u/x/xamyzhao/anaconda3/envs/cuda92/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "/afs/csail.mit.edu/u/x/xamyzhao/anaconda3/envs/cuda92/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "/afs/csail.mit.edu/u/x/xamyzhao/anaconda3/envs/cuda92/lib/python3.6/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "/afs/csail.mit.edu/u/x/xamyzhao/anaconda3/envs/cuda92/lib/python3.6/imp.py", line 243, in load_module
    return load_dynamic(name, filename, file)
  File "/afs/csail.mit.edu/u/x/xamyzhao/anaconda3/envs/cuda92/lib/python3.6/imp.py", line 343, in load_dynami

Load training dataset

In [ ]:
dataset_key = 'adni-3d-100-unnorm-masked-csts2'
import vte_runner
data_params = vte_runner.named_vte_data_params[dataset_key]
    
ds = adni_loader.ADNIDataset(data_params)

vol_shape = tuple(data_params['img_shape'])

# just load some examples so we can get the image size, but actually use a generator later...
(X_unlabeled, _, ids_unlabeled),\
(X_labeled_train, Y_labeled_train, ids_labeled_train), \
(X_labeled_valid, Y_labeled_valid, ids_labeled_valid), \
label_mapping \
= ds.load_dataset(debug=False)

In [ ]:
print(X_unlabeled.shape)
print(X_labeled_train.shape)
print(X_labeled_valid.shape)
print(ds.files_labeled_train)
print(ds.params['load_vols'])
import IPython
import PIL
IPython.display.display(PIL.Image.fromarray((X_labeled_train[0, :, :, 64, 0]*255).astype(np.uint8)))
IPython.display.display(PIL.Image.fromarray((X_unlabeled[0, :, :, 64, 0]*255).astype(np.uint8)))


In [ ]:
K.clear_session()
start_iter = 0
voxelmorph_model = load_model(
    #'/afs/csail.mit.edu/u/x/xamyzhao/voxelmorph/models/vm2_cc.h5',
    './experiments/voxelmorph/vm2_cc_AtoUMS_newdataset_iter100000.h5',#.format(start_iter),
    custom_objects={'Dense3DSpatialTransformer': Dense3DSpatialTransformer,
                   'SpatialTransformer': nrn_layers.SpatialTransformer,
                   },
    compile=False,
)

# create diffeomorphic model
sys.path.append('../voxelmorph')
import src.networks as vm_networks
start_iter = 0
nf_enc = [16, 32, 32, 32]
nf_dec = [32, 32, 32, 32, 16, 3]
vm_diffeo_model = vm_networks.miccai2018_bidir(
    (160, 192, 224),
    [16, 32, 32, 32],
    nf_dec,
    bidir=True,
    indexing='xy'
)


In [ ]:
vm_diffeo_model.summary()
import data_utils
import src.losses as vm_losses

# just train voxelmorph
vm_diffeo_model.compile(
    #loss=['mean_squared_error', vm_losses.gradientLoss('l2')],
    #loss=[vm_losses.cc3D(), vm_losses.gradientLoss('l2')],
    loss=[vm_losses.NCC().loss, vm_losses.NCC().loss, vm_losses.Grad('l2').loss, 'mean_squared_error'],
          #vm_losses.gradientLoss('l2')],
    #loss_weights=[1.0, ,0.01],
    loss_weights=[1., 1., 1., 0.],#0.01],
    optimizer=Adam(0.0001)
)



In [ ]:
n_convs = 0
# copy weights from regular voxelmorph as initialization
for li, l in enumerate(vm_diffeo_model.layers):
    print(l.name)
    
    if l.name == voxelmorph_model.layers[li].name:
        
        if n_convs < 5:
            print(l.name)
            vm_diffeo_model.layers[li].set_weights(voxelmorph_model.layers[li].get_weights())

    if 'conv' in l.name.lower():
        n_convs += 1
    
    
        

In [ ]:
print(ds.files_labeled_train[0])
# source_X = ds.X_atlas
source_X, _ = adni_loader._load_vol_and_seg(ds.files_labeled_train[0], load_seg=False, mask_vol=ds.params['masked'])
source_X = source_X[np.newaxis]
IPython.display.display(PIL.Image.fromarray((source_X[0, :, :, 64, 0]*255).astype(np.uint8)))



In [ ]:
print(start_iter)
n_train_iters = 100000
vol_gen = ds.gen_vols_batch(['labeled_train', 'unlabeled_train'], batch_size=1, randomize=True)
print(ds.files_labeled_train + ds.files_unlabeled_train)

target_X, _ = next(vol_gen)
zeros_flow = np.zeros(target_X.shape[:-1] + (3,))

for bi in range(n_train_iters + 1):
    
    target_X, _ = next(vol_gen)
    vm_losses = vm_diffeo_model.train_on_batch([source_X, target_X], [target_X, source_X, zeros_flow, zeros_flow])
    print('Iter {}, loss {}'.format(bi, vm_losses))
    
    if bi > 0 and bi % 2000 == 0:
        vm_diffeo_model.save('./experiments/voxelmorph/vm2_cc_AtoUMS_100k_CStoUMS_ij_bidir_init5_iter{}.h5'.format(start_iter + bi))

In [ ]:
# # save a voxelmorph wrapper
# import sys
# sys.path.append('../voxelmorph-sandbox')
# import voxelmorph.networks as vm_networks
# import tensorflow as tf
# from voxelmorph import dense_3D_spatial_transformer
# from keras.models import load_model
 
# sys.path.append('../neuron')
# import neuron.layers as nrn_layers
# import neuron.utils as nrn_utils
# sys.path.append('../voxelmorph-sandbox')
# import voxelmorph.networks as vm_networks
# from voxelmorph.dense_3D_spatial_transformer import Dense3DSpatialTransformer

                                
# vm_diffeo_model = load_model(
#     #'/afs/csail.mit.edu/u/x/xamyzhao/voxelmorph/models/vm2_cc.h5',
#     './experiments/voxelmorph/vm2_cc_AtoUMS_newdataset_100k_bidir_iter10000.h5',#.format(start_iter),
#     custom_objects={'Dense3DSpatialTransformer': dense_3D_spatial_transformer.Dense3DSpatialTransformer, 
#                     'interp_upsampling': vm_networks.interp_upsampling,
#                     'meshgrid': vm_networks.meshgrid,
#                     'tf': tf,
                    
#                     'VecInt': nrn_layers.VecInt,
#                     'SpatialTransformer': nrn_layers.SpatialTransformer,
#                     'nrn_utils': nrn_utils,
#                     'nrn_layers': nrn_layers,
#                    },
#     compile=False,
# )

# from keras.layers import Input, Lambda
# from keras.models import Model

# vol_shape = (160, 192, 224, 1)
# input_src = Input(vol_shape)
# input_tgt = Input(vol_shape)

# warped, backwarped, _ = vm_diffeo_model([input_src, input_tgt])
# flow = vm_diffeo_model.get_layer('diffflow').output

# wrapper_model = Model(inputs=[input_src, input_tgt], outputs=[flow, warped], name='vmmiccai_bidir_cc_wrapper')
# wrapper_model.summary()
# wrapper_model.save('./experiments/voxelmorph/vm2_cc_AtoUMS_newdataset_100k_bidir_iter10000_wrapper.h5')